In [1]:
"""Headers"""

from __future__ import print_function
from PIL import Image
import os
import os.path
import numpy as np
import sys
import math
import time
import pickle
import numpy.random

import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity

import csv
import copy
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os.path
import sys
import torch
import torch.utils.data
import torchvision
import torchvision.transforms as transforms

import cv2

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from boxplot import *
from coco_dict import *

In [2]:
from datasetloader import *
# ---------------------

train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                ])
test_transform = None
# ---------------------
st_time = time.time()

#boxParams = [[8, 0.5], [8, 1.], [8, 2], [16, 0.5], [16, 1.], [16, 2.], [32, 0.5], [32, 1.], [32, 2.]]
boxParams = [[16, 0.5], [16, 1.], [16, 2], [32, 0.5], [32, 1.], [32, 2.], [64, 0.5], [64, 1.], [64, 2.]]
#boxParams = [[32, 0.5], [32, 1.], [32, 2], [64, 0.5], [64, 1.], [64, 2.], [128, 0.5], [128, 1.], [128, 2.]]

TRAIN_BS = 1

trainset = DataSet(fold="train", transform=train_transform, max_img = 100, boxParams = boxParams, maxDim = 300)

print(trainset.data.shape)

trainLoader = torch.utils.data.DataLoader(trainset, batch_size=TRAIN_BS,
                                          shuffle=True, num_workers=0)
print("Ttrain set size: "+str(len(trainset)))

valset = DataSet(fold="valid", transform=train_transform, max_img = 20, boxParams = boxParams)

valLoader = torch.utils.data.DataLoader(valset, batch_size=TRAIN_BS,
                                          shuffle=True, num_workers=0)
print("Val set size: "+str(len(valset)))

print('Loading Dataset took: ', time.time() - st_time, ' seconds')

(101,)
Ttrain set size: 101
Val set size: 21
Loading Dataset took:  87.77257585525513  seconds


In [ ]:
from model import *
IS_GPU = False

#boxParams = torch.tensor([[8, 0.5], [8, 1.], [8, 2], [16, 0.5], [16, 1.], [16, 2.], [32, 0.5], [32, 1.], [32, 2.]])
boxParams = torch.tensor(boxParams)

rcnn = PreTrainedResNet(feature_extracting = True, model = "resnet18")

#rcnn.load('Full_200')


if IS_GPU:
    rcnn = PreTrainedResNet(feature_extracting = True, model = "resnet18", IS_GPU = True)
    rcnn = rcnn.cuda()

rcnn.train(trainLoader, valLoader, boxParams, NUM_EPOCHS = 20, class_start = 20)

Epoch  0  : Time required ->  9.295088529586792
Epoch  1  : Time required ->  9.21955394744873
Epoch  2  : Time required ->  9.211281061172485
Epoch  3  : Time required ->  9.302090406417847
Epoch  4  : Time required ->  9.244795560836792
Epoch  5  : Time required ->  9.321896076202393


In [ ]:
rcnn.train(trainLoader, valLoader, boxParams, NUM_EPOCHS = 50, class_start = -1)

In [ ]:
plt.plot(rcnn.valLoss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Validation Set Loss")

In [ ]:
plt.plot(rcnn.trainLoss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train Set Loss")

In [ ]:
plt.plot(rcnn.fastLoss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("fast RCNN Loss")

In [ ]:
plt.plot(rcnn.rpnLoss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("RPN Loss")

In [ ]:
it = iter(trainLoader)
img, target = it.next()
rcnn.predict(img, cutoff = 0.5, boxParams = boxParams, dummyLabel = True)

In [ ]:
rcnn.predict(img, cutoff = 0.8, boxParams = boxParams, dummyLabel = True)

In [ ]:
rcnn.predict(img, cutoff = 0.9, boxParams = boxParams, secondLabel = True)

In [ ]:
rcnn.predict(img, cutoff = 0.7, boxParams = boxParams, dummyLabel = False)

In [ ]:
rcnn.save('Full_200')